Make predictions with model saved by **gnn_50M_100M_03.ipynb**.

In [ ]:
!pip install pgzip
!pip install rdkit
!pip install torch_geometric
!pip install torch-scatter
!pip install lightning

In [ ]:
import os

import pandas as pd
import numpy as np

import rdkit
from rdkit import Chem

import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader

from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_scatter import scatter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import CSVLogger

from sklearn.metrics import average_precision_score

from multiprocessing import Pool

import _pickle as  cPickle
import bz2
import pgzip
from tqdm import tqdm

print('import ok!')

In [4]:
def save_compressed_pickle(file, data, type = "pgzip", n_treads = 1):
    if type == "pgzip":
        with pgzip.open(file , 'w', thread = n_treads) as f:
            cPickle.dump(data, f)
    elif type == "bz2":
        with bz2.BZ2File(file , 'w') as f:
            cPickle.dump(data, f)
    else:
        print("unsupported")

def load_decompress_pickle(file, type = "pgzip", n_treads = 4):
    if type == "pgzip":
        data = pgzip.open(file, thread = n_treads)
        data = cPickle.load(data)
    elif type == "bz2":
        data = bz2.BZ2File(file, 'rb')
        data = cPickle.load(data)
    else:
        print("unsupported")
    return data

def compute_ap(df):
    df = pd.concat(
        [df.loc[:, ["BRD4", "HSA", "sEH", "label_id", "split"]].melt(
            id_vars = ["label_id", "split"],
            var_name = "protein",
            value_name = "y_true"
            ),
        df.loc[:, ["BRD4_pred", "HSA_pred", "sEH_pred", "label_id", "split"]].melt(
            id_vars = ["label_id", "split"],
            var_name = "_",
            value_name = "y_pred"
            ).drop("split", axis = 1)],
        axis = 1
    )

    ap = df.groupby(["split", "protein"]).apply(
        lambda x: average_precision_score(x.y_true, x.y_pred)
        #include_groups = True
    ).reset_index(name = "AP")

    return(ap)

In [7]:
TEST_FILE = "../data/test_ensemble_wide_replace_dy.parquet"
TEST_GRAPH_FILE = "../data/test_graph_2M.gz"

SUBMIT_FILE = "../data/submit_replace_dy.parquet"
SUBMIT_GRAPH_FILE = "../data/submit_graph.gz"

# Set your checkpoint path
CHECKPOINT_PATH = "../GNN_100M_second_half/fold_0_epoch_1_step_15796_0.0106.ckpt"

POS_WEIGHT = torch.tensor([1, 1, 1])

LR = 5e-4
BATCH_SIZE = 5000
ACCUMULATE_GRAD_BATCHES = 1
EPOCHS = 10
OUT_DIR = ""
SEED = 42
NUM_WORKERS = 1

os.makedirs("../submits", exist_ok = True)

In [8]:
test_graph = load_decompress_pickle(TEST_GRAPH_FILE, type = "pgzip", n_treads = 2)

In [10]:
df = pd.read_parquet(TEST_FILE)
df["split"] = 0
df["id"] = df.reset_index().index
df

,molecule_smiles,BRD4,HSA,sEH,split,id
0,CNC(=O)c1ccc(Nc2nc(NCC3CCN(c4ccc(Br)cc4)C3)nc(...,0,0,0,0,0
1,C#CCOc1cccc(CNc2nc(Nc3c(C(=O)OC)c[nH]c3C(=O)OC...,0,0,0,0,1
2,CNC(=O)C1CN(c2nc(NCc3cccc(N4CCOCC4)c3)nc(NCC3(...,0,0,0,0,2
3,CNC(=O)[C@@H]1CCCCN1c1nc(NCCNC(=O)c2ccno2)nc(N...,0,0,0,0,3
4,CNC(=O)c1ccc2nc(Nc3nc(NCc4nnc(SC)s4)nc(Nc4ccc(...,0,0,0,0,4
...,...,...,...,...,...,...
1999995,CNC(=O)c1cc(Nc2nc(NCC(F)(F)C(N)=O)nc(Nc3cc(O)c...,0,0,0,0,1999995
1999996,CNC(=O)c1cccc(I)c1Nc1nc(NCCCCC2(C)OCCO2)nc(NCC...,0,0,0,0,1999996
1999997,CNC(=O)c1cccc(C(=O)O)c1Nc1nc(NCc2cn[nH]c2)nc(N...,0,0,0,0,1999997
1999998,CNC(=O)c1ccc(Nc2nc(Nc3ccc(C)nc3Cl)nc(Nc3c(O)nc...,0,0,0,0,1999998


In [ ]:
# helper
# torch version of np unpackbits
#https://gist.github.com/vadimkantorov/30ea6d278bc492abf6ad328c6965613a

def tensor_dim_slice(tensor, dim, dim_slice):
    return tensor[(dim if dim >= 0 else dim + tensor.dim()) * (slice(None),) + (dim_slice,)]

# @torch.jit.script
def packshape(shape, dim: int = -1, mask: int = 0b00000001, dtype=torch.uint8, pack=True):
    dim = dim if dim >= 0 else dim + len(shape)
    bits, nibble = (
        8 if dtype is torch.uint8 else 16 if dtype is torch.int16 else 32 if dtype is torch.int32 else 64 if dtype is torch.int64 else 0), (
        1 if mask == 0b00000001 else 2 if mask == 0b00000011 else 4 if mask == 0b00001111 else 8 if mask == 0b11111111 else 0)
    # bits = torch.iinfo(dtype).bits # does not JIT compile
    assert nibble <= bits and bits % nibble == 0
    nibbles = bits // nibble
    shape = (shape[:dim] + (int(math.ceil(shape[dim] / nibbles)),) + shape[1 + dim:]) if pack else (
                shape[:dim] + (shape[dim] * nibbles,) + shape[1 + dim:])
    return shape, nibbles, nibble

# @torch.jit.script
def F_unpackbits(tensor, dim: int = -1, mask: int = 0b00000001, shape=None, out=None, dtype=torch.uint8):
    dim = dim if dim >= 0 else dim + tensor.dim()
    shape_, nibbles, nibble = packshape(tensor.shape, dim=dim, mask=mask, dtype=tensor.dtype, pack=False)
    shape = shape if shape is not None else shape_
    out = out if out is not None else torch.empty(shape, device=tensor.device, dtype=dtype)
    assert out.shape == shape

    if shape[dim] % nibbles == 0:
        shift = torch.arange((nibbles - 1) * nibble, -1, -nibble, dtype=torch.uint8, device=tensor.device)
        shift = shift.view(nibbles, *((1,) * (tensor.dim() - dim - 1)))
        return torch.bitwise_and((tensor.unsqueeze(1 + dim) >> shift).view_as(out), mask, out=out)

    else:
        for i in range(nibbles):
            shift = nibble * i
            sliced_output = tensor_dim_slice(out, dim, slice(i, None, nibbles))
            sliced_input = tensor.narrow(dim, 0, sliced_output.shape[dim])
            torch.bitwise_and(sliced_input >> shift, mask, out=sliced_output)
    return out

class dotdict(dict):
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)


print('helper ok!')

In [12]:
# mol to graph adopted from
# from https://github.com/LiZhang30/GPCNDTA/blob/main/utils/DrugGraph.py

PACK_NODE_DIM=9
PACK_EDGE_DIM=1
NODE_DIM=40#PACK_NODE_DIM*8
EDGE_DIM=PACK_EDGE_DIM*8

def one_of_k_encoding(x, allowable_set, allow_unk=False):
    if x not in allowable_set:
        if allow_unk:
            x = allowable_set[-1]
        else:
            raise Exception(f'input {x} not in allowable set{allowable_set}!!!')
    return list(map(lambda s: x == s, allowable_set))


#Get features of an atom (one-hot encoding:)
'''
    1.atom element: 44+1 dimensions
    2.the atom's hybridization: 5 dimensions
    3.degree of atom: 6 dimensions
    4.total number of H bound to atom: 6 dimensions
    5.number of implicit H bound to atom: 6 dimensions
    6.whether the atom is on ring: 1 dimension
    7.whether the atom is aromatic: 1 dimension
    Total: 70 dimensions
'''

ATOM_SYMBOL = ['B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'S', 'Si']
#print('ATOM_SYMBOL', len(ATOM_SYMBOL))44

HYBRIDIZATION_TYPE = [
    Chem.rdchem.HybridizationType.S,
    Chem.rdchem.HybridizationType.SP,
    Chem.rdchem.HybridizationType.SP2,
    Chem.rdchem.HybridizationType.SP3,
    Chem.rdchem.HybridizationType.SP3D
]

def get_atom_feature(atom):
    feature = (
         one_of_k_encoding(atom.GetSymbol(), ATOM_SYMBOL)
       + one_of_k_encoding(atom.GetHybridization(), HYBRIDIZATION_TYPE)
       + one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5])
       + one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5])
       + one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5])
       + [atom.IsInRing()]
       + [atom.GetIsAromatic()]
    )
    #feature = np.array(feature, dtype=np.uint8)
    feature = np.packbits(feature)
    return feature


#Get features of an edge (one-hot encoding)
'''
    1.single/double/triple/aromatic: 4 dimensions
    2.the atom's hybridization: 1 dimensions
    3.whether the bond is on ring: 1 dimension
    Total: 6 dimensions
'''

def get_bond_feature(bond):
    bond_type = bond.GetBondType()
    feature = [
        bond_type == Chem.rdchem.BondType.SINGLE,
        bond_type == Chem.rdchem.BondType.DOUBLE,
        bond_type == Chem.rdchem.BondType.TRIPLE,
        bond_type == Chem.rdchem.BondType.AROMATIC,
        bond.GetIsConjugated(),
        bond.IsInRing()
    ]
    #feature = np.array(feature, dtype=np.uint8)
    feature = np.packbits(feature)
    return feature


def smile_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    N = mol.GetNumAtoms()
    node_feature = []
    edge_feature = []
    edge = []
    for i in range(mol.GetNumAtoms()):
        atom_i = mol.GetAtomWithIdx(i)
        atom_i_features = get_atom_feature(atom_i)
        node_feature.append(atom_i_features)

        for j in range(mol.GetNumAtoms()):
            bond_ij = mol.GetBondBetweenAtoms(i, j)
            if bond_ij is not None:
                edge.append([i, j])
                bond_features_ij = get_bond_feature(bond_ij)
                edge_feature.append(bond_features_ij)
    node_feature=np.stack(node_feature)
    edge_feature=np.stack(edge_feature)
    edge = np.array(edge,dtype=np.uint8)
    return N,edge,node_feature,edge_feature

def to_pyg_format(N,edge,node_feature,edge_feature):
    graph = Data(
        idx=-1,
        edge_index = torch.from_numpy(edge.T).int(),
        x          = torch.from_numpy(node_feature).byte(),
        edge_attr  = torch.from_numpy(edge_feature).byte(),
    )
    return graph


In [ ]:
#debug one example
g = to_pyg_format(*smile_to_graph(smiles="C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1"))
print(g)
print('[Dy] is replaced by C !!')
print('smile_to_graph() ok!')

In [14]:
class SmilesDataset(Dataset):
    def __init__(
        self,
        df,
        target_cols = ["BRD4", "HSA", "sEH"],
        mode = "train"
    ):
        self.df = df
        self.target_cols = target_cols
        self.mode = mode
    def __len__(self):
        """
        Length of dataset.
        """
        return len(self.df)

    def __getitem__(self, index):
        """
        Get one item.
        """
        if self.mode != "test":
            X, y, label_ids, split = self.__data_generation(index)
            return X, y, label_ids, split
        else:
            X = self.__data_generation(index)
            return X

    def __data_generation(self, index):

        row = self.df.iloc[index]
        id = torch.tensor(row.id, dtype = torch.float32)
        split = torch.tensor(row.split, dtype = torch.float32)

        if self.mode != "test":
            y = row[self.target_cols].to_numpy().astype(np.float32)
            y = torch.tensor(y, dtype = torch.float32)

        X = row.molecule_smiles
        X = to_pyg_format(*smile_to_graph(X))

        if self.mode != "test":
            return X, y, id, split
        else:
            return X

class SmilesDataset(Dataset):
    def __init__(
        self,
        df,
        graph,
        target_cols = ["BRD4", "HSA", "sEH"],
        mode = "train"
    ):
        self.df = df
        self.graph = graph
        self.target_cols = target_cols
        self.mode = mode
    def __len__(self):
        """
        Length of dataset.
        """
        return len(self.df)

    def __getitem__(self, index):
        """
        Get one item.
        """
        if self.mode != "test":
            X, y, label_ids, split = self.__data_generation(index)
            return X, y, label_ids, split
        else:
            X = self.__data_generation(index)
            return X

    def __data_generation(self, index):

        row = self.df.iloc[index]
        id = torch.tensor(row.id, dtype = torch.float32)
        split = torch.tensor(row.split, dtype = torch.float32)

        if self.mode != "test":
            y = row[self.target_cols].to_numpy().astype(np.float32)
            y = torch.tensor(y, dtype = torch.float32)

        X = to_pyg_format(*self.graph[index])

        if self.mode != "test":
            return X, y, id, split
        else:
            return X

In [15]:
#MODEL: simple MPNNModel
#from https://github.com/chaitjo/geometric-gnn-dojo/blob/main/geometric_gnn_101.ipynb

# i have removed all comments here to jepp it clean. refer to orginal link for code comments
# of MPNNModel
class MPNNLayer(MessagePassing):
    def __init__(self, emb_dim=64, edge_dim=4, aggr='add'):
        super().__init__(aggr=aggr)

        self.emb_dim = emb_dim
        self.edge_dim = edge_dim
        self.mlp_msg = nn.Sequential(
            nn.Linear(2 * emb_dim + edge_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU(),
            nn.Linear(emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU()
        )
        self.mlp_upd = nn.Sequential(
            nn.Linear(2 * emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU(),
            nn.Linear(emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU()
        )

    def forward(self, h, edge_index, edge_attr):
        out = self.propagate(edge_index, h=h, edge_attr=edge_attr)
        return out

    def message(self, h_i, h_j, edge_attr):
        msg = torch.cat([h_i, h_j, edge_attr], dim=-1)
        return self.mlp_msg(msg)

    def aggregate(self, inputs, index):
        return scatter(inputs, index, dim=self.node_dim, reduce=self.aggr)

    def update(self, aggr_out, h):
        upd_out = torch.cat([h, aggr_out], dim=-1)
        return self.mlp_upd(upd_out)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(emb_dim={self.emb_dim}, aggr={self.aggr})')


class MPNNModel(nn.Module):
    def __init__(self, num_layers=4, emb_dim=64, in_dim=11, edge_dim=4, out_dim=1):
        super().__init__()

        self.lin_in = nn.Linear(in_dim, emb_dim)

        # Stack of MPNN layers
        self.convs = torch.nn.ModuleList()
        for layer in range(num_layers):
            self.convs.append(MPNNLayer(emb_dim, edge_dim, aggr='add'))

        self.pool = global_mean_pool

    def forward(self, data): #PyG.Data - batch of PyG graphs

        h = self.lin_in(F_unpackbits(data.x,-1).float())

        for conv in self.convs:
            h = h + conv(h, data.edge_index.long(), F_unpackbits(data.edge_attr,-1).float())  # (n, d) -> (n, d)

        h_graph = self.pool(h, data.batch)
        return h_graph

In [16]:
class EMATracker:
    def __init__(self, alpha: float = 0.05):
        super().__init__()
        self.alpha = alpha
        self._value = None

    def update(self, new_value):
        if self._value is None:
            self._value = new_value
        else:
            self._value = (
                new_value * self.alpha +
                self._value * (1-self.alpha)
            )

    @property
    def value(self):
        return self._value

class Net(L.LightningModule):
    def __init__(self, lr, epochs, steps_per_epoch, lr_scheduler = False, fold = 0):
        super().__init__()
        self.lr = lr
        self.epochs = epochs
        self.steps_per_epoch = steps_per_epoch
        self.lr_scheduler = lr_scheduler
        self.fold = fold

        self.output_type = ['infer', 'loss']

        self.graph_dim = 96

        self.smile_encoder = MPNNModel(
            in_dim=NODE_DIM,
            edge_dim=EDGE_DIM,
            emb_dim=self.graph_dim,
            num_layers=4
        )

        self.bind = nn.Sequential(
            nn.Linear(self.graph_dim, 1024),
            #nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(1024, 1024),
            #nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 3)
        )

        self.loss = nn.BCEWithLogitsLoss(pos_weight = POS_WEIGHT)
        self.validation_step_outputs = []

    def forward(self, x):
        x = self.smile_encoder(x)
        out = self.bind(x)
        return out

    def training_step(self, batch, batch_idx):
        x, y, label_ids, split = batch
        out = self.forward(x)
        loss = self.loss(out, y)
        self.log("train_loss", loss, on_step = True, on_epoch = True,
                 prog_bar = True, batch_size = BATCH_SIZE)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y, label_ids, split = batch
        out = self.forward(x)
        val_loss = self.loss(out, y)
        self.log("val_loss", val_loss, on_step = False, on_epoch = True,
                 logger = True, prog_bar = True, batch_size = BATCH_SIZE)
        self.validation_step_outputs.append(
            {"val_loss": val_loss, "predictions": out, "targets": y, "label_ids": label_ids, "split": split}
        )
        return {"val_loss": val_loss}

    def on_validation_end(self):
        outputs = self.validation_step_outputs
        output_val = torch.cat([x['predictions'] for x in outputs], dim = 0)
        output_val = F.sigmoid(output_val).to(torch.float32).cpu().detach().numpy()
        target_val = torch.cat([x['targets'] for x in outputs], dim = 0).to(torch.float32).cpu().detach().numpy()
        label_ids = torch.cat([x['label_ids'] for x in outputs], dim = 0).to(torch.float32).cpu().detach().numpy()
        split = torch.cat([x['split'] for x in outputs], dim = 0).to(torch.float32).cpu().detach().numpy()
        self.validation_step_outputs = []

        TARGETS = ["BRD4", "HSA", "sEH"]
        PREDS = [f"{t}_pred" for t in TARGETS]
        val_df = pd.DataFrame(target_val, columns = list(TARGETS))
        pred_df = pd.DataFrame(output_val, columns = list(PREDS))
        val_df = pd.concat([val_df, pred_df], axis = 1)
        val_df["label_id"] = label_ids
        val_df["split"] = split

        ap = compute_ap(val_df)
        ap["epoch"] = self.current_epoch
        ap["step"] = self.global_step
        ap.loc[ap.split == 1, "split"] = "val"
        ap.loc[ap.split == 2, "split"] = "val_nonshare"
        ap.to_csv(f"{OUT_DIR}/metrics.csv", index = False, mode = "a",
                  header = not os.path.exists(f"{OUT_DIR}/metrics.csv"))

        map = ap.AP.mean()

        print(f"validation MAP: {map}")

        val_df.to_csv(f"{OUT_DIR}/val_df_fold_{self.fold}_epoch_{self.current_epoch}_step_{self.global_step}_{map:.4f}.csv",
                      index = False)


    def predict_step(self, batch, batch_idx, dataloader_idx = 0):
        return F.sigmoid(self(batch))

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr = self.lr,
            betas = (0.9, 0.999),
            eps = 1e-08,
            weight_decay = 0.01
        )
        if self.lr_scheduler:
            scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                self.lr,
                total_steps =  math.floor(self.steps_per_epoch * self.epochs / ACCUMULATE_GRAD_BATCHES)
            ),
            "interval": "step",
        }
            return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}
        else:
            return optimizer

# Test

In [17]:
test_ds = SmilesDataset(df, test_graph, mode = "test")
test_loader = PyGDataLoader(
    test_ds,
    shuffle = False,
    batch_size = 1024
)

In [18]:
model = Net.load_from_checkpoint(
    CHECKPOINT_PATH,
    lr = LR,
    epochs = EPOCHS,
    steps_per_epoch = 1,
    lr_scheduler = False
)
model = model.eval()

In [ ]:
preds = []
with torch.inference_mode():
    for i, test_batch in enumerate(tqdm(test_loader)):
        test_batch = test_batch.to("cuda")
        out = model(test_batch)
        pred = torch.sigmoid(out).cpu().numpy()
        preds.append(pred)

preds = np.concatenate(preds)


In [20]:
pred_cols = ["BRD4_pred", "HSA_pred", "sEH_pred"]
df[pred_cols] = preds

In [21]:
df["split"] = "test"
df

,molecule_smiles,BRD4,HSA,sEH,split,id,BRD4_pred,HSA_pred,sEH_pred
0,CNC(=O)c1ccc(Nc2nc(NCC3CCN(c4ccc(Br)cc4)C3)nc(...,0,0,0,test,0,5.268890e-03,1.843924e-04,1.012409e-04
1,C#CCOc1cccc(CNc2nc(Nc3c(C(=O)OC)c[nH]c3C(=O)OC...,0,0,0,test,1,5.103975e-04,1.687295e-03,3.444607e-06
2,CNC(=O)C1CN(c2nc(NCc3cccc(N4CCOCC4)c3)nc(NCC3(...,0,0,0,test,2,7.399669e-06,1.516384e-06,2.822315e-06
3,CNC(=O)[C@@H]1CCCCN1c1nc(NCCNC(=O)c2ccno2)nc(N...,0,0,0,test,3,4.596250e-07,1.096087e-04,4.246950e-07
4,CNC(=O)c1ccc2nc(Nc3nc(NCc4nnc(SC)s4)nc(Nc4ccc(...,0,0,0,test,4,2.469472e-02,9.267978e-03,4.524392e-05
...,...,...,...,...,...,...,...,...,...
1999995,CNC(=O)c1cc(Nc2nc(NCC(F)(F)C(N)=O)nc(Nc3cc(O)c...,0,0,0,test,1999995,4.472955e-05,1.137288e-04,8.819937e-07
1999996,CNC(=O)c1cccc(I)c1Nc1nc(NCCCCC2(C)OCCO2)nc(NCC...,0,0,0,test,1999996,3.050507e-07,1.819548e-05,9.333046e-06
1999997,CNC(=O)c1cccc(C(=O)O)c1Nc1nc(NCc2cn[nH]c2)nc(N...,0,0,0,test,1999997,1.356074e-06,7.613175e-06,2.142304e-09
1999998,CNC(=O)c1ccc(Nc2nc(Nc3ccc(C)nc3Cl)nc(Nc3c(O)nc...,0,0,0,test,1999998,7.404792e-04,3.643142e-03,1.265051e-06


In [2]:
def compute_ap(df):
    df = pd.concat(
        [df.loc[:, ["BRD4", "HSA", "sEH", "split"]].melt(
            id_vars = "split",
            var_name = "protein",
            value_name = "y_true"
            ),
        df.loc[:, ["BRD4_pred", "HSA_pred", "sEH_pred", "split"]].melt(
            id_vars = "split",
            var_name = "_",
            value_name = "y_pred"
            ).drop("split", axis = 1)],
        axis = 1
    )

    ap = df.groupby(["split", "protein"]).apply(
        lambda x: average_precision_score(x.y_true, x.y_pred)
    ).reset_index(name = "AP")

    return(ap)

compute_ap(df)

,split,protein,AP
0,test,BRD4,0.600890
1,test,HSA,0.405091
2,test,sEH,0.902702


In [23]:
df.drop(["id", "split"], axis = 1)
df.to_parquet("gnn_50M_100M_test_pred.parquet")
df

,molecule_smiles,BRD4,HSA,sEH,split,id,BRD4_pred,HSA_pred,sEH_pred
0,CNC(=O)c1ccc(Nc2nc(NCC3CCN(c4ccc(Br)cc4)C3)nc(...,0,0,0,test,0,5.268890e-03,1.843924e-04,1.012409e-04
1,C#CCOc1cccc(CNc2nc(Nc3c(C(=O)OC)c[nH]c3C(=O)OC...,0,0,0,test,1,5.103975e-04,1.687295e-03,3.444607e-06
2,CNC(=O)C1CN(c2nc(NCc3cccc(N4CCOCC4)c3)nc(NCC3(...,0,0,0,test,2,7.399669e-06,1.516384e-06,2.822315e-06
3,CNC(=O)[C@@H]1CCCCN1c1nc(NCCNC(=O)c2ccno2)nc(N...,0,0,0,test,3,4.596250e-07,1.096087e-04,4.246950e-07
4,CNC(=O)c1ccc2nc(Nc3nc(NCc4nnc(SC)s4)nc(Nc4ccc(...,0,0,0,test,4,2.469472e-02,9.267978e-03,4.524392e-05
...,...,...,...,...,...,...,...,...,...
1999995,CNC(=O)c1cc(Nc2nc(NCC(F)(F)C(N)=O)nc(Nc3cc(O)c...,0,0,0,test,1999995,4.472955e-05,1.137288e-04,8.819937e-07
1999996,CNC(=O)c1cccc(I)c1Nc1nc(NCCCCC2(C)OCCO2)nc(NCC...,0,0,0,test,1999996,3.050507e-07,1.819548e-05,9.333046e-06
1999997,CNC(=O)c1cccc(C(=O)O)c1Nc1nc(NCc2cn[nH]c2)nc(N...,0,0,0,test,1999997,1.356074e-06,7.613175e-06,2.142304e-09
1999998,CNC(=O)c1ccc(Nc2nc(Nc3ccc(C)nc3Cl)nc(Nc3c(O)nc...,0,0,0,test,1999998,7.404792e-04,3.643142e-03,1.265051e-06


# Submit

In [24]:
test_graph = load_decompress_pickle(SUBMIT_GRAPH_FILE, type = "pgzip", n_treads = 2)

In [44]:
df = pd.read_parquet(SUBMIT_FILE)
df = df[["molecule_smiles"]].drop_duplicates()
df["split"] = 0
df["id"] = df.reset_index().index
df = df.reset_index(drop = True)

In [38]:
test_ds = SmilesDataset(df, test_graph, mode = "test")
test_loader = PyGDataLoader(
    test_ds,
    shuffle = False,
    batch_size = 5000
)

In [ ]:
preds = []
with torch.inference_mode():
    for i, test_batch in enumerate(tqdm(test_loader)):
        test_batch = test_batch.to("cuda")
        out = model(test_batch)
        pred = torch.sigmoid(out).cpu().numpy()
        preds.append(pred)

preds = np.concatenate(preds)

In [45]:
submit = pd.DataFrame(preds, columns = ["BRD4", "HSA", "sEH"])
submit = submit.reset_index(drop = True)

submit = pd.concat([df, submit], axis = 1)

submit

,molecule_smiles,split,id,BRD4,HSA,sEH
0,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,0,0,6.186083e-06,0.000640,3.347678e-07
1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,0,1,8.244032e-06,0.000723,4.295103e-07
2,C#CCCC[C@H](Nc1nc(NCC2(O)CCCC2(C)C)nc(Nc2ccc(C...,0,2,3.736051e-06,0.000633,8.588781e-07
3,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2sc(Cl)c...,0,3,5.454234e-06,0.000178,7.578028e-08
4,C#CCCC[C@H](Nc1nc(NCC2CCC(SC)CC2)nc(Nc2ccc(C=C...,0,4,3.391129e-06,0.000103,8.999456e-08
...,...,...,...,...,...,...
878017,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(Nc2ccc3c(cnn...,0,878017,2.358707e-06,0.000084,1.789188e-08
878018,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(NCC2CCC3CC3C...,0,878018,1.848727e-07,0.000007,1.192215e-09
878019,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(Nc2cc(C(=O)O...,0,878019,8.786287e-07,0.000008,6.000774e-09
878020,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(NCCC2CCC(OC)...,0,878020,4.310131e-06,0.000017,3.172965e-08


In [46]:
df = pd.read_parquet(SUBMIT_FILE)
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295246830,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,BRD4
1,295246831,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,HSA
2,295246832,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,sEH
3,295246833,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,BRD4
4,295246834,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,HSA
...,...,...,...,...,...,...
1674891,296921721,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(NCCC2CCC(OC)...,HSA
1674892,296921722,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(NCCC2CCC(OC)...,sEH
1674893,296921723,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(NCc2cccs2)nc...,BRD4
1674894,296921724,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,CNC(=O)[C@H](CCCN=[N+]=[N-])Nc1nc(NCc2cccs2)nc...,HSA


In [47]:
submit = pd.melt(
    submit,
    id_vars = ["molecule_smiles"],
    value_vars = ["BRD4", "HSA", "sEH"],
    value_name = "binds",
    var_name = "protein_name"
)
submit = pd.merge(
    df,
    submit,
    how = "outer",
    on = ["molecule_smiles", "protein_name"]
)
submit = submit[["id", "binds"]]

submit[submit["binds"].isna()] = 0
submit = submit.dropna()

submit = submit.drop_duplicates()

In [48]:
submit

,id,binds
0,295246830.0,6.186083e-06
1,295246831.0,6.399485e-04
2,295246832.0,3.347678e-07
3,295246833.0,8.244032e-06
4,295246834.0,7.231326e-04
...,...,...
1674891,296921721.0,1.657991e-05
1674892,296921722.0,3.172965e-08
1674893,296921723.0,5.090506e-07
1674894,296921724.0,4.610884e-05


In [49]:
submit.to_csv("../submits/gnn_50M_100M.csv", index = False)